In [6]:
# 1. Add models for student, subject and student_subject from previous lessons in SQLAlchemy.

import mysql.connector
from mysql.connector import Error
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
# from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class Student(Base):
    __tablename__ = "students"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(16))
    age = Column(Integer)
    subjects = relationship("Student_subject", back_populates="student")

class Subject(Base):
    __tablename__ = "subjects"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(16))
    students = relationship("Student_subject", back_populates="subject")

class Student_subject(Base):
    __tablename__ = "student_subject"
    
    serial_id = Column(Integer, primary_key=True, autoincrement=True)
    student_id = Column(Integer, ForeignKey("students.id"))
    subject_id = Column(Integer, ForeignKey("subjects.id"))

    student = relationship("Student", back_populates="subjects")
    subject = relationship("Subject", back_populates="students")

# have issues with Postgre installation :( 
mysql_config = {
    "host": "localhost",
    #"database": "university",   
    "user": "root", 
    "password": "1111"
}

database = "university"

# Database and tables
try:
    connection = mysql.connector.connect(**mysql_config)
    cursor = connection.cursor()
    
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}") #.format(mysql_config['database']))
    print("Database OK")
    
    mysql_config["database"] = database
    connection.database = mysql_config["database"]
    
    engine = create_engine(f"mysql+mysqlconnector://{mysql_config['user']}:{mysql_config['password']}@{mysql_config['host']}/{mysql_config['database']}")
    
    Base.metadata.create_all(engine)
    print("Tables OK")
    
except Error as e:
    print("Error:", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()

# Insert some data
Session = sessionmaker(bind=engine)
session = Session()

student1 = Student(name="Alice", age = 18)
student2 = Student(name="Cooper", age = 21)
student3 = Student(name="Luis", age = 25)
student4 = Student(name="Carol", age = 36)

subject1 = Subject(name="English")
subject2 = Subject(name="Chemitry")
subject3 = Subject(name="Science")
subject4 = Subject(name="Math")

student_subject1 = Student_subject(student=student1, subject=subject1)
student_subject2 = Student_subject(student=student1, subject=subject2)
student_subject3 = Student_subject(student=student2, subject=subject4)
student_subject4 = Student_subject(student=student2, subject=subject3)
student_subject5 = Student_subject(student=student4, subject=subject1)
student_subject6 = Student_subject(student=student3, subject=subject4)
student_subject7 = Student_subject(student=student3, subject=subject1)

# INSERT data
session.add_all([student1, student2, student3, student4, subject1, subject2, subject3, subject4])
session.add_all([student_subject1, student_subject2, student_subject3, student_subject4, student_subject5, student_subject6, student_subject7])
session.commit()

print("Insert OK")



Database OK
Tables OK
Insert OK


In [7]:
# 2. Find all students` name that visited 'English' classes.

eng_std = session.query(Student).join(Student_subject).join(Subject).filter(Subject.name == 'English').all()
[print(student.name) for student in eng_std]

# 3. (Optional): Rewrite queries from the previous lesson using SQLAlchemy.


all_students = session.query(Student).all()

# all students by name;
student_names = session.query(Student.name).all()

# students age < 20;
students_under_20 = session.query(Student.name).filter(Student.age < 20).all()

# age between 10 and 20 years;
students_between_10_and_20 = session.query(Student.name).filter(Student.age > 10, Student.age < 20).all()

# age < 20 or name - Alice ;
students_under_20_or_alice = session.query(Student.name).filter((Student.age < 20) | (Student.name == 'Alice')).all()

# order by age;
students_ordered_by_age = session.query(Student.name, Student.age).order_by(Student.age).all()

# DESC age < 30 ;
students_age_30_ordered_desc = session.query(Student.name, Student.age).filter(Student.age <= 30).order_by(Student.age.desc()).all()


# unpacking, for example
print("\n")
[print(student.name, student.age) for student in all_students]


# SELECT * FROM students INNER JOIN student_subject ON students.id = student_subject.student_id;
result_inner_join = session.query(Student, Student_subject).filter(Student.id == Student_subject.student_id).all()

# SELECT students.name, student_subject.subject_id FROM students INNER JOIN student_subject ON students.id = student_subject.student_id;
result_select_columns = session.query(Student.name, Student_subject.subject_id).filter(Student.id == Student_subject.student_id).all()

# SELECT * FROM students INNER JOIN student_subject ON students.id = student_subject.student_id INNER JOIN subjects ON subjects.id = student_subject.subject_id;
result_three_table_join = session.query(Student, Student_subject, Subject).filter(Student.id == Student_subject.student_id, Student_subject.subject_id == Subject.id).all()

# SELECT students.id, subjects.id FROM students INNER JOIN subjects ON subjects.id = student.id;
result_subquery_join = session.query(Student.id, Subject.id).filter(Student.id == Subject.id).all()

# SELECT * FROM students FULL JOIN student_subject ON students.id = student_subject.student_id;
result_full_join = session.query(Student, Student_subject).filter(Student.id == Student_subject.student_id).all()

# SELECT * FROM students LEFT JOIN student_subject ON students.id = student_subject.student_id;
result_left_join = session.query(Student, Student_subject).filter(Student.id == Student_subject.student_id).all()

# SELECT * FROM students RIGHT JOIN student_subject ON students.id = student_subject.student_id;
result_right_join = session.query(Student, Student_subject).filter(Student.id == Student_subject.student_id).all()

# unpacking, for example
print("\n")
for std, std_sbj, sbj in result_three_table_join:
    print ("ID: {} Name: {} Serial_id: {} Subject : {}".format(std.id, std.name, std_sbj.serial_id, sbj.name)) # if sbj.name == "Math" else "" 


session.close()


Alice
Carol
Luis
Alice
Carol
Luis


Alice 18
Carol 36
Luis 25
Cooper 21
Alice 18
Carol 36
Luis 25
Cooper 21


ID: 1 Name: Alice Serial_id: 1 Subject : English
ID: 1 Name: Alice Serial_id: 7 Subject : Chemitry
ID: 2 Name: Carol Serial_id: 2 Subject : English
ID: 3 Name: Luis Serial_id: 3 Subject : English
ID: 3 Name: Luis Serial_id: 4 Subject : Math
ID: 4 Name: Cooper Serial_id: 5 Subject : Math
ID: 4 Name: Cooper Serial_id: 6 Subject : Science
ID: 5 Name: Alice Serial_id: 8 Subject : English
ID: 5 Name: Alice Serial_id: 14 Subject : Chemitry
ID: 6 Name: Carol Serial_id: 9 Subject : English
ID: 7 Name: Luis Serial_id: 10 Subject : English
ID: 7 Name: Luis Serial_id: 11 Subject : Math
ID: 8 Name: Cooper Serial_id: 12 Subject : Math
ID: 8 Name: Cooper Serial_id: 13 Subject : Science
